In [205]:
%load_ext autoreload
%autoreload 2

import numpy as np
from ts.supplier import TickSupplier
from ts.supplier import BarSupplier, Bar
from ts.supplier import BarAggregation
from ts.supplier import BarFeatureSupplier, BarFeatures
from ts.supplier import MultiplexSupplier

filepath = "/Users/jacobgora/projects/data/continuous_futures/CME-HO.parquet"
tick_supplier = TickSupplier(instrument="CME-HO")
tick_supplier.from_parquet(filepath)

bar_supplier_hundred = BarSupplier(
    supplier=tick_supplier,
    bar_aggregation=BarAggregation.TIME_MINUTES,
    size=100
)

bar_supplier_ten_thousand = BarSupplier(
    supplier=tick_supplier,
    bar_aggregation=BarAggregation.TIME_MINUTES,
    size=10_000
)

bar_features_supplier = BarFeatureSupplier(supplier=bar_supplier_hundred)
suppliers = [bar_features_supplier, bar_supplier_hundred, bar_supplier_ten_thousand]


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [266]:
def z_score(col, over):
    return ((pl.col(col) - pl.mean(col).over(over)) / pl.std(col).over(over)).suffix("_z_score")



In [293]:
df = bar_supplier_ten_thousand.data
df = df.with_columns(
    (
        df['bar-CME-HO-time_minutes-10000-timestamp'].dt.hour() + " " +
        df['bar-CME-HO-time_minutes-10000-timestamp'].dt.minute() + " " +
        df['bar-CME-HO-time_minutes-10000-timestamp'].dt.day()
    ).alias("timehash")
)


In [283]:
pl.mean(pl.col("bar-CME-HO-time_minutes-10000-volume"))

ValueError: Axis should be 0 or 1.

In [278]:
df['bar-CME-HO-time_minutes-10000-timestamp'].dt.day()

shape: (156,)
Series: 'bar-CME-HO-time_minutes-10000-timestamp' [u32]
[
	9
	16
	23
	30
	6
	13
	20
	27
	3
	10
	17
	24
	...
	28
	4
	9
	16
	23
	30
	7
	14
	21
	28
	4
	11
	16
]

In [275]:
df

bar-CME-HO-time_minutes-10000-open,bar-CME-HO-time_minutes-10000-low,bar-CME-HO-time_minutes-10000-high,bar-CME-HO-time_minutes-10000-close,bar-CME-HO-time_minutes-10000-volume,bar-CME-HO-time_minutes-10000-timestamp,bar-CME-HO-time_minutes-10000-timedelta,bar-CME-HO-time_minutes-10000-ask_size,bar-CME-HO-time_minutes-10000-bid_size,bar-CME-HO-time_minutes-10000-return,timehash
f64,f64,f64,f64,i64,"datetime[ns, US/Eastern]",i64,i64,i64,f64,str
18853.0,18668.0,19725.0,19397.0,42757,2019-12-09 16:58:53 EST,597533,21073,21684,0.0,"""16 58 9"""
19420.0,19157.0,20102.0,20033.0,50521,2019-12-16 16:14:47 EST,598487,25683,24838,0.032789,"""16 14 16"""
20039.0,20017.0,20408.0,20252.0,110296,2019-12-23 14:59:55.006 EST,599875,58029,52267,0.010932,"""14 59 23"""
20251.0,20202.0,20782.0,20416.0,86977,2019-12-30 13:39:49.003 EST,599989,44523,42454,0.008098,"""13 39 30"""
20415.0,20162.0,21056.0,20307.0,145506,2020-01-06 12:19:56.002 EST,599996,73062,72444,-0.005339,"""12 19 6"""
20310.0,19052.0,21195.0,19117.0,208188,2020-01-13 10:59:58 EST,599995,103952,104236,-0.0586,"""10 59 13"""
19120.0,18430.0,19224.0,18785.0,152107,2020-01-20 09:39:54 EST,599994,74530,77577,-0.017367,"""9 39 20"""
18785.0,16617.0,18835.0,16817.0,179117,2020-01-27 08:19:59 EST,599999,88610,90507,-0.104764,"""8 19 27"""
16817.0,15986.0,17313.0,16072.0,214878,2020-02-03 06:59:57 EST,599997,105046,109832,-0.0443,"""6 59 3"""


In [ ]:
df.with_columns(z_score('bar-CME-HO-time_minutes-10000-volume', 'timehash'))

In [ ]:
df.groupby_dynamic('bar-CME-HO-time_minutes-10000-timestamp', every="1y").agg(z_score('bar-CME-HO-time_minutes-10000-volume', 'timehash'))

bar-CME-HO-time_minutes-10000-open,bar-CME-HO-time_minutes-10000-low,bar-CME-HO-time_minutes-10000-high,bar-CME-HO-time_minutes-10000-close,bar-CME-HO-time_minutes-10000-volume,bar-CME-HO-time_minutes-10000-timestamp,bar-CME-HO-time_minutes-10000-timedelta,bar-CME-HO-time_minutes-10000-ask_size,bar-CME-HO-time_minutes-10000-bid_size,bar-CME-HO-time_minutes-10000-return,timehash,bar-CME-HO-time_minutes-10000-volume_z_score
f64,f64,f64,f64,i64,"datetime[ns, US/Eastern]",i64,i64,i64,f64,str,f64
18853.0,18668.0,19725.0,19397.0,42757,2019-12-09 16:58:53 EST,597533,21073,21684,0.0,"""16 58""",-1.259921
19420.0,19157.0,20102.0,20033.0,50521,2019-12-16 16:14:47 EST,598487,25683,24838,0.032789,"""16 14""",NaN
20039.0,20017.0,20408.0,20252.0,110296,2019-12-23 14:59:55.006 EST,599875,58029,52267,0.010932,"""14 59""",-0.707107
20251.0,20202.0,20782.0,20416.0,86977,2019-12-30 13:39:49.003 EST,599989,44523,42454,0.008098,"""13 39""",-0.865432
20415.0,20162.0,21056.0,20307.0,145506,2020-01-06 12:19:56.002 EST,599996,73062,72444,-0.005339,"""12 19""",0.305656
20310.0,19052.0,21195.0,19117.0,208188,2020-01-13 10:59:58 EST,599995,103952,104236,-0.0586,"""10 59""",0.988409
19120.0,18430.0,19224.0,18785.0,152107,2020-01-20 09:39:54 EST,599994,74530,77577,-0.017367,"""9 39""",-0.191266
18785.0,16617.0,18835.0,16817.0,179117,2020-01-27 08:19:59 EST,599999,88610,90507,-0.104764,"""8 19""",1.142215
16817.0,15986.0,17313.0,16072.0,214878,2020-02-03 06:59:57 EST,599997,105046,109832,-0.0443,"""6 59""",1.134097


InvalidOperationError: window expression not allowed in aggregation

In [265]:
df.groupby_rolling('bar-CME-HO-time_minutes-10000-timestamp', period="1d").agg(z_score('bar-CME-HO-time_minutes-10000-timestamp', 'timehash'))

InvalidOperationError: window expression not allowed in aggregation

In [ ]:
df.select(z_score("C", ["A", "B"]))

In [249]:
df = pl.DataFrame({
    "A": [1,1,1,1,2,2,],
    "B": ["a", "a", "b", "b", "a", "b"],
    "C": range(10, 16),
})


In [250]:
df

A,B,C
i64,str,i64
1,"""a""",10
1,"""a""",11
1,"""b""",12
1,"""b""",13
2,"""a""",14
2,"""b""",15


In [247]:


#.groupby#.agg(pl.col("Close").mean())


AttributeError: 'DynamicGroupBy' object has no attribute 'apply'

In [237]:
(
    df['bar-CME-HO-time_minutes-10000-timestamp'].dt.hour() + " " +
    df['bar-CME-HO-time_minutes-10000-timestamp'].dt.minute()
).alias("timehash")

shape: (156,)
Series: 'timehash' [str]
[
	"16 58"
	"16 14"
	"14 59"
	"13 39"
	"12 19"
	"10 59"
	"9 39"
	"8 19"
	"6 59"
	"5 39"
	"4 19"
	"2 59"
	...
	"19 58"
	"18 39"
	"16 59"
	"16 59"
	"16 59"
	"16 59"
	"16 59"
	"16 59"
	"16 59"
	"16 59"
	"16 59"
	"16 59"
	"16 59"
]

shape: (156,)
Series: 'bar-CME-HO-time_minutes-10000-timestamp' [u32]
[
	16
	16
	14
	13
	12
	10
	9
	8
	6
	5
	4
	2
	...
	19
	18
	16
	16
	16
	16
	16
	16
	16
	16
	16
	16
	16
]

In [226]:
df['bar-CME-HO-time_minutes-10000-timestamp'].dt.weekday()

shape: (156,)
Series: 'bar-CME-HO-time_minutes-10000-timestamp' [u32]
[
	0
	0
	0
	0
	0
	0
	0
	0
	0
	0
	0
	0
	...
	6
	6
	4
	4
	4
	4
	4
	4
	4
	4
	4
	4
	2
]

In [ ]:
pl.col("Date").dt.year()

In [ ]:
df['bar-CME-HO-time_minutes-10000-timestamp'].apply(lambda dt: pl.Series([dt.daya, dt.hour]))

In [223]:
df['bar-CME-HO-time_minutes-10000-timestamp'].apply(lambda dt: pl.Series([dt.day, dt.hour]))

shape: (156,)
Series: 'bar-CME-HO-time_minutes-10000-timestamp' [list]
[
	[9, 16]
	[16, 16]
	[23, 14]
	[30, 13]
	[6, 12]
	[13, 10]
	[20, 9]
	[27, 8]
	[3, 6]
	[10, 5]
	[17, 4]
	[24, 2]
	...
	[28, 19]
	[4, 18]
	[9, 16]
	[16, 16]
	[23, 16]
	[30, 16]
	[7, 16]
	[14, 16]
	[21, 16]
	[28, 16]
	[4, 16]
	[11, 16]
	[16, 16]
]

In [222]:
df['bar-CME-HO-time_minutes-10000-timestamp'].apply(lambda dt: pl.Struct([dt.day, dt.hour]))

thread '<unnamed>' panicked at 'a formatting trait implementation returned an error: Error', library/alloc/src/fmt.rs:612:32


PanicException: a formatting trait implementation returned an error: Error

In [212]:
df[['bar-CME-HO-time_minutes-10000-TIMESTAMP']]

NotFoundError: bar-CME-HO-time_minutes-10000-TIMESTAMP

In [199]:
multiplex_supplier = MultiplexSupplier(suppliers=suppliers)

In [200]:
multiplex_supplier.instruments

['CME-HO']

In [ ]:
multiplex_supplier.data

AttributeError: 'MultiplexSupplier' object has no attribute 'dat'

In [202]:
multiplex_supplier.bar_features

['bar_features-bar-CME-HO-time_minutes-100-return_timedelta',
 'bar_features-bar-CME-HO-time_minutes-100-bid_size_timedelta',
 'bar_features-bar-CME-HO-time_minutes-100-ask_size_timedelta',
 'bar_features-bar-CME-HO-time_minutes-100-volume_delta',
 'bar_features-bar-CME-HO-time_minutes-100-size',
 'bar_features-bar-CME-HO-time_minutes-100-ofi',
 'bar_features-bar-CME-HO-time_minutes-100-ofi_normalized',
 'bar_features-bar-CME-HO-time_minutes-100-open_high',
 'bar_features-bar-CME-HO-time_minutes-100-open_low',
 'bar_features-bar-CME-HO-time_minutes-100-open_close',
 'bar_features-bar-CME-HO-time_minutes-100-internal_bar_strength']

In [197]:
multiplex_supplier.data.columns

['bar-CME-HO-time_minutes-100-open',
 'bar-CME-HO-time_minutes-100-low',
 'bar-CME-HO-time_minutes-100-high',
 'bar-CME-HO-time_minutes-100-close',
 'bar-CME-HO-time_minutes-100-volume',
 'bar-CME-HO-time_minutes-100-timestamp',
 'bar-CME-HO-time_minutes-100-timedelta',
 'bar-CME-HO-time_minutes-100-ask_size',
 'bar-CME-HO-time_minutes-100-bid_size',
 'bar-CME-HO-time_minutes-100-return',
 'bar_features-bar-CME-HO-time_minutes-100-return_timedelta',
 'bar_features-bar-CME-HO-time_minutes-100-bid_size_timedelta',
 'bar_features-bar-CME-HO-time_minutes-100-ask_size_timedelta',
 'bar_features-bar-CME-HO-time_minutes-100-volume_delta',
 'bar_features-bar-CME-HO-time_minutes-100-size',
 'bar_features-bar-CME-HO-time_minutes-100-ofi',
 'bar_features-bar-CME-HO-time_minutes-100-ofi_normalized',
 'bar_features-bar-CME-HO-time_minutes-100-open_high',
 'bar_features-bar-CME-HO-time_minutes-100-open_low',
 'bar_features-bar-CME-HO-time_minutes-100-open_close',
 'bar_features-bar-CME-HO-time_minut

In [137]:
aggregation_types = [e.bar_aggregation for e in suppliers]
if not all([agg_type == suppliers[0].bar_aggregation for agg_type in aggregation_types]):
    raise RuntimeError(f"Require common BarAggregation type. Passed: {aggregation_types}")

aggregation_sizes = [e.size for e in suppliers]
min_aggregation_size = min(aggregation_sizes)
if not all([(e % min_aggregation_size) == 0 for e in aggregation_sizes]):
    raise RuntimeError(f"BarAggregation sizes need to be integer factor of highest frequency supplier.")


df = suppliers[np.argmin(aggregation_sizes)].data.clone()
print("parent alias", suppliers[np.argmin(aggregation_sizes)].alias)
index_col = suppliers[np.argmin(aggregation_sizes)].index
for pos, supplier in enumerate(suppliers):

    if pos == np.argmin(aggregation_sizes):
        continue
    print(supplier.alias, supplier.index)
    #print(supplier.data[supplier.index].head(2))
    df = df.join_asof(supplier.data, left_on=index_col, right_on=supplier.index)#.drop(f"{supplier.index}_right")

#bar_features_supplier

parent alias bar_features-bar-CME-HO-time_minutes-100
bar-CME-HO-time_minutes-100 bar-CME-HO-time_minutes-100-timestamp
bar-CME-HO-time_minutes-10000 bar-CME-HO-time_minutes-10000-timestamp


DuplicateError: Cannot do hstack operation. Column with name: timestamp_right already exists

In [155]:

bar_supplier_hundred = BarSupplier(
    supplier=tick_supplier,
    bar_aggregation=BarAggregation.TIME_MINUTES,
    size=100
)

In [156]:
bar_supplier_hundred.data.columns

['bar-CME-HO-time_minutes-100-open',
 'bar-CME-HO-time_minutes-100-low',
 'bar-CME-HO-time_minutes-100-high',
 'bar-CME-HO-time_minutes-100-close',
 'bar-CME-HO-time_minutes-100-volume',
 'bar-CME-HO-time_minutes-100-timestamp',
 'bar-CME-HO-time_minutes-100-timedelta',
 'bar-CME-HO-time_minutes-100-ask_size',
 'bar-CME-HO-time_minutes-100-bid_size',
 'bar-CME-HO-time_minutes-100-return']

In [157]:
tick_supplier.data.columns

['timestamp', 'price', 'quantity', 'side', '__index_level_0__']

In [158]:
for supplier in suppliers:
    print(f"columns = {[e for e in supplier.data.columns if 'timestamp' in e]}")
    print(f"{supplier.alias = }")
    print(f"{supplier.index = }")
    print(supplier.data.columns)

columns = ['bar-CME-HO-time_minutes-100-timestamp']
supplier.alias = 'bar_features-bar-CME-HO-time_minutes-100'
supplier.index = 'bar-CME-HO-time_minutes-100-timestamp'
['bar-CME-HO-time_minutes-100-open', 'bar-CME-HO-time_minutes-100-low', 'bar-CME-HO-time_minutes-100-high', 'bar-CME-HO-time_minutes-100-close', 'bar-CME-HO-time_minutes-100-volume', 'bar-CME-HO-time_minutes-100-timestamp', 'bar-CME-HO-time_minutes-100-timedelta', 'bar-CME-HO-time_minutes-100-ask_size', 'bar-CME-HO-time_minutes-100-bid_size', 'bar-CME-HO-time_minutes-100-return', 'bar_features-bar-CME-HO-time_minutes-100-return_timedelta', 'bar_features-bar-CME-HO-time_minutes-100-bid_size_timedelta', 'bar_features-bar-CME-HO-time_minutes-100-ask_size_timedelta', 'bar_features-bar-CME-HO-time_minutes-100-volume_delta', 'bar_features-bar-CME-HO-time_minutes-100-size', 'bar_features-bar-CME-HO-time_minutes-100-ofi', 'bar_features-bar-CME-HO-time_minutes-100-ofi_normalized', 'bar_features-bar-CME-HO-time_minutes-100-open_h

['timestamp',
 'bar-CME-HO-time_minutes-10000-open',
 'bar-CME-HO-time_minutes-10000-low',
 'bar-CME-HO-time_minutes-10000-high',
 'bar-CME-HO-time_minutes-10000-close',
 'bar-CME-HO-time_minutes-10000-volume',
 'bar-CME-HO-time_minutes-10000-timestamp',
 'bar-CME-HO-time_minutes-10000-timedelta',
 'bar-CME-HO-time_minutes-10000-ask_size',
 'bar-CME-HO-time_minutes-10000-bid_size',
 'bar-CME-HO-time_minutes-10000-return']

In [160]:

suppliers = [suppliers[i] for i in np.argsort(aggregation_sizes)]
df = suppliers[0].data
index_col = suppliers[0].index
for supplier in suppliers[1:]:
    df = df.join_asof(supplier.data, left_on=index_col, right_on=supplier.index)
    #print(f"{supplier.index =}")
    #print([e for e in df.columns if "" in e])


In [161]:
df

bar-CME-HO-time_minutes-100-open,bar-CME-HO-time_minutes-100-low,bar-CME-HO-time_minutes-100-high,bar-CME-HO-time_minutes-100-close,bar-CME-HO-time_minutes-100-volume,bar-CME-HO-time_minutes-100-timestamp,bar-CME-HO-time_minutes-100-timedelta,bar-CME-HO-time_minutes-100-ask_size,bar-CME-HO-time_minutes-100-bid_size,bar-CME-HO-time_minutes-100-return,bar_features-bar-CME-HO-time_minutes-100-return_timedelta,bar_features-bar-CME-HO-time_minutes-100-bid_size_timedelta,bar_features-bar-CME-HO-time_minutes-100-ask_size_timedelta,bar_features-bar-CME-HO-time_minutes-100-volume_delta,bar_features-bar-CME-HO-time_minutes-100-size,bar_features-bar-CME-HO-time_minutes-100-ofi,bar_features-bar-CME-HO-time_minutes-100-ofi_normalized,bar_features-bar-CME-HO-time_minutes-100-open_high,bar_features-bar-CME-HO-time_minutes-100-open_low,bar_features-bar-CME-HO-time_minutes-100-open_close,bar_features-bar-CME-HO-time_minutes-100-internal_bar_strength,bar-CME-HO-time_minutes-100-open_right,bar-CME-HO-time_minutes-100-low_right,bar-CME-HO-time_minutes-100-high_right,bar-CME-HO-time_minutes-100-close_right,bar-CME-HO-time_minutes-100-volume_right,bar-CME-HO-time_minutes-100-timedelta_right,bar-CME-HO-time_minutes-100-ask_size_right,bar-CME-HO-time_minutes-100-bid_size_right,bar-CME-HO-time_minutes-100-return_right,bar-CME-HO-time_minutes-10000-open,bar-CME-HO-time_minutes-10000-low,bar-CME-HO-time_minutes-10000-high,bar-CME-HO-time_minutes-10000-close,bar-CME-HO-time_minutes-10000-volume,bar-CME-HO-time_minutes-10000-timestamp,bar-CME-HO-time_minutes-10000-timedelta,bar-CME-HO-time_minutes-10000-ask_size,bar-CME-HO-time_minutes-10000-bid_size,bar-CME-HO-time_minutes-10000-return
f64,f64,f64,f64,i64,"datetime[ns, US/Eastern]",i64,i64,i64,f64,f64,f64,f64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,i64,i64,f64,f64,f64,f64,f64,i64,"datetime[ns, US/Eastern]",i64,i64,i64,f64
18853.0,18843.0,18868.0,18868.0,15,2019-12-02 20:09:38 EST,4178,7,8,0.0,0.0,0.001915,0.001675,-1,15,1,0.066667,0.999205,1.000531,0.999205,1.0,18853.0,18843.0,18868.0,18868.0,15,4178,7,8,0.0,null,null,null,null,null,null,null,null,null,null
18851.0,18850.0,18875.0,18875.0,12,2019-12-02 21:32:53 EST,3611,7,5,0.000371,1.0274e-7,0.001385,0.001939,2,12,-2,-0.166667,0.998728,1.000053,0.998728,1.0,18851.0,18850.0,18875.0,18875.0,12,3611,7,5,0.000371,null,null,null,null,null,null,null,null,null,null
18870.0,18859.0,18875.0,18859.0,15,2019-12-02 23:05:56 EST,3943,10,5,-0.000848,-2.1498e-7,0.001268,0.002536,5,15,-5,-0.333333,0.999735,1.000583,1.000583,0.0,18870.0,18859.0,18875.0,18859.0,15,3943,10,5,-0.000848,null,null,null,null,null,null,null,null,null,null
18860.0,18858.0,18881.0,18868.0,23,2019-12-03 01:17:13 EST,5690,13,10,0.000477,8.3871e-8,0.001757,0.002285,3,23,-3,-0.130435,0.998888,1.000106,0.999576,0.434783,18860.0,18858.0,18881.0,18868.0,23,5690,13,10,0.000477,null,null,null,null,null,null,null,null,null,null
18867.0,18858.0,18905.0,18897.0,150,2019-12-03 02:59:59 EST,5979,87,63,0.001537,2.5707e-7,0.010537,0.014551,24,150,-24,-0.16,0.99799,1.000477,0.998412,0.829787,18867.0,18858.0,18905.0,18897.0,150,5979,87,63,0.001537,null,null,null,null,null,null,null,null,null,null
18895.0,18859.0,18951.0,18903.0,442,2019-12-03 04:35:15 EST,5715,274,168,0.000318,5.5557e-8,0.029396,0.047944,106,442,-106,-0.239819,0.997045,1.001909,0.999577,0.478261,18895.0,18859.0,18951.0,18903.0,442,5715,274,168,0.000318,null,null,null,null,null,null,null,null,null,null
18890.0,18774.0,18917.0,18795.0,220,2019-12-03 06:19:29.003 EST,5959,120,100,-0.005713,-9.5878e-7,0.016781,0.020138,20,220,-20,-0.090909,0.998573,1.006179,1.005055,0.146853,18890.0,18774.0,18917.0,18795.0,220,5959,120,100,-0.005713,null,null,null,null,null,null,null,null,null,null
18790.0,18734.0,18832.0,18739.0,364,2019-12-03 07:59:54 EST,5976,189,175,-0.00298,-4.9858e-7,0.029284,0.031627,14,364,-14,-0.038462,0.99777,1.002989,1.002722,0.05102,18790.0,18734.0,18832.0,18739.0,364,5976,189,175,-0.00298,null,null,null,null,null,null,nul

In [125]:
supplier.data.columns

['timestamp',
 'bar-CME-HO-time_minutes-10000-open',
 'bar-CME-HO-time_minutes-10000-low',
 'bar-CME-HO-time_minutes-10000-high',
 'bar-CME-HO-time_minutes-10000-close',
 'bar-CME-HO-time_minutes-10000-volume',
 'bar-CME-HO-time_minutes-10000-timestamp',
 'bar-CME-HO-time_minutes-10000-timedelta',
 'bar-CME-HO-time_minutes-10000-ask_size',
 'bar-CME-HO-time_minutes-10000-bid_size',
 'bar-CME-HO-time_minutes-10000-return']

In [111]:
index_col

'bar-CME-HO-time_minutes-100-timestamp'

In [78]:
bar_supplier_hundred.data[bar_supplier_hundred.index]

shape: (10988,)
Series: 'bar-CME-HO-time_minutes-100-timestamp' [datetime[ns, US/Eastern]]
[
	2019-12-02 20:09:38 EST
	2019-12-02 21:32:53 EST
	2019-12-02 23:05:56 EST
	2019-12-03 01:17:13 EST
	2019-12-03 02:59:59 EST
	2019-12-03 04:35:15 EST
	2019-12-03 06:19:29.003 EST
	2019-12-03 07:59:54 EST
	2019-12-03 09:39:53.002 EST
	2019-12-03 11:19:56 EST
	2019-12-03 12:59:42.001 EST
	2019-12-03 14:39:50.002 EST
	...
	2022-11-15 20:36:52.115 EST
	2022-11-15 22:31:23.689 EST
	2022-11-15 23:37:57.627005 EST
	2022-11-16 01:47:54.898 EST
	2022-11-16 03:39:51.373 EST
	2022-11-16 05:15:02.963 EST
	2022-11-16 06:54:55.144 EST
	2022-11-16 08:39:57.088 EST
	2022-11-16 10:19:38.499 EST
	2022-11-16 11:59:58.963 EST
	2022-11-16 13:39:59.872 EST
	2022-11-16 15:19:42.165001 EST
	2022-11-16 16:59:59.952 EST
]

In [73]:
bar_supplier_ten_thousand.data[bar_supplier_ten_thousand.index]

shape: (156,)
Series: 'bar-CME-HO-time_minutes-10000-timestamp' [datetime[ns, US/Eastern]]
[
	2019-12-09 16:58:53 EST
	2019-12-16 16:14:47 EST
	2019-12-23 14:59:55.006 EST
	2019-12-30 13:39:49.003 EST
	2020-01-06 12:19:56.002 EST
	2020-01-13 10:59:58 EST
	2020-01-20 09:39:54 EST
	2020-01-27 08:19:59 EST
	2020-02-03 06:59:57 EST
	2020-02-10 05:39:18 EST
	2020-02-17 04:19:24.001 EST
	2020-02-24 02:59:47 EST
	...
	2022-08-28 19:58:34.987001 EDT
	2022-09-04 18:39:09.101001 EDT
	2022-09-09 16:59:36.624 EDT
	2022-09-16 16:59:30.200002 EDT
	2022-09-23 16:59:58.015 EDT
	2022-09-30 16:59:55.195 EDT
	2022-10-07 16:59:55.965 EDT
	2022-10-14 16:59:55.221 EDT
	2022-10-21 16:59:55.282 EDT
	2022-10-28 16:59:55.704 EDT
	2022-11-04 16:59:55.509 EDT
	2022-11-11 16:59:44.157001 EST
	2022-11-16 16:59:59.952 EST
]

In [69]:
suppliers[np.argmin(aggregation_sizes)].alias

'bar_features-bar-CME-HO-time_minutes-100'

In [28]:
bar_features_supplier

'BarFeatureSupplier'

In [31]:
bar_supplier.bar_aggregation

'volume'

In [ ]:
.groupby_dynamic(TradeTick.TIMESTAMP, every=f"{size}ms")

In [12]:
(
    data
    .agg()

)

NameError: name 'data' is not defined

In [10]:
bar_supplier.data.groupby_dynamic("timestamp", every=f"10ms")

In [ ]:

feat_supplier = BarFeatureSupplier(bar_supplier)

In [13]:
bar_supplier.data[Bar.LOW]

low
f64
18843.0
18869.0
18881.0
18898.0
18873.0
18859.0
18881.0
18793.0
18774.0


In [14]:
feat_supplier.data.head()

open,low,high,close,volume,timestamp,timedelta,ask_size,bid_size,return,return_timedelta,bid_size_timedelta,ask_size_timedelta,volume_delta,size,ofi,ofi_normalized,open_high,open_low,open_close,internal_bar_strength
f64,f64,f64,f64,i64,"datetime[ns, US/Eastern]",i64,i64,i64,f64,f64,f64,f64,i64,i64,i64,f64,f64,f64,f64,f64
18853.0,18843.0,18882.0,18876.0,99,2019-12-03 02:10:48 EST,25848,59,40,0.0,0.0,0.001548,0.002283,19,99,-19,-0.191919,0.998464,1.000531,0.998782,0.846154
18873.0,18869.0,18905.0,18898.0,100,2019-12-03 02:57:47.002 EST,2818,53,47,0.001166,4.1359e-7,0.016678,0.018808,6,100,-6,-0.06,0.998307,1.000212,0.998677,0.805556
18898.0,18881.0,18913.0,18912.0,100,2019-12-03 03:18:35.001 EST,1247,78,22,0.000741,5.9408e-7,0.017642,0.06255,56,100,-56,-0.56,0.999207,1.0009,0.99926,0.96875
18912.0,18898.0,18934.0,18899.0,100,2019-12-03 03:28:48.001 EST,612,82,18,-0.000687,-0.000001,0.029412,0.133987,64,100,-64,-0.64,0.998838,1.000741,1.000688,0.027778
18899.0,18873.0,18910.0,18903.0,100,2019-12-03 03:31:41 EST,172,74,26,0.000212,0.000001,0.151163,0.430233,48,100,-48,-0.48,0.999418,1.001378,0.999788,0.810811


In [10]:
feat_supplier.data

open,low,high,close,volume,timestamp,timedelta,ask_size,bid_size,return,return_timedelta,bid_size_timedelta,ask_size_timedelta,volume_delta,size,ofi,ofi_normalized,open_high,open_low,open_close,internal_bar_strength
f64,f64,f64,f64,i64,"datetime[ns, US/Eastern]",i64,i64,i64,f64,f64,f64,f64,i64,i64,i64,f64,f64,f64,f64,f64
18853.0,18843.0,18882.0,18876.0,99,2019-12-03 02:10:48 EST,25848,59,40,0.0,0.0,0.001548,0.002283,19,99,-19,-0.191919,0.998464,1.000531,0.998782,0.846154
18873.0,18869.0,18905.0,18898.0,100,2019-12-03 02:57:47.002 EST,2818,53,47,0.001166,4.1359e-7,0.016678,0.018808,6,100,-6,-0.06,0.998307,1.000212,0.998677,0.805556
18898.0,18881.0,18913.0,18912.0,100,2019-12-03 03:18:35.001 EST,1247,78,22,0.000741,5.9408e-7,0.017642,0.06255,56,100,-56,-0.56,0.999207,1.0009,0.99926,0.96875
18912.0,18898.0,18934.0,18899.0,100,2019-12-03 03:28:48.001 EST,612,82,18,-0.000687,-0.000001,0.029412,0.133987,64,100,-64,-0.64,0.998838,1.000741,1.000688,0.027778
18899.0,18873.0,18910.0,18903.0,100,2019-12-03 03:31:41 EST,172,74,26,0.000212,0.000001,0.151163,0.430233,48,100,-48,-0.48,0.999418,1.001378,0.999788,0.810811
18903.0,18859.0,18940.0,18940.0,99,2019-12-03 04:02:02.005 EST,1817,30,69,0.001957,0.000001,0.037975,0.016511,-39,99,39,0.393939,0.998046,1.002333,0.998046,1.0
18940.0,18881.0,18951.0,18881.0,100,2019-12-03 05:10:29 EST,4106,42,58,-0.003115,-7.5867e-7,0.014126,0.010229,-16,100,16,0.16,0.99942,1.003125,1.003125,0.0
18876.0,18793.0,18878.0,18803.0,101,2019-12-03 05:30:24 EST,1173,62,39,-0.004131,-0.000004,0.033248,0.052856,23,101,-23,-0.227723,0.999894,1.004417,1.003882,0.117647
18805.0,18774.0,18835.0,18797.0,100,2019-12-03 06:59:11 EST,5320,50,50,-0.000319,-5.9981e-8,0.009398,0.009398,0,100,0,0.0,0.998407,1.001651,1.000426,0.377049


feat_supplier

In [114]:
(
    feat_supplier.data
    .with_column(
        (pl.col(BarFeatures.OFI))
    )
)

shape: (190945,)
Series: 'ofi' [i64]
[
	-19
	-6
	-56
	-64
	-48
	39
	16
	-23
	0
	-22
	-25
	17
	...
	-18
	-22
	0
	4
	38
	-14
	-44
	-88
	42
	2
	32
	4
	14
]

In [104]:
feat_supplier.data

open,low,high,close,volume,timestamp,timedelta,ask_size,bid_size,return,return_timedelta,bid_size_timedelta,ask_size_timedelta,volume_delta,size,ofi,ofi_normalized,open_high,open_low,open_close,internal_bar_strength
f64,f64,f64,f64,i64,"datetime[ns, US/Eastern]",i64,i64,i64,f64,f64,f64,f64,i64,i64,i64,f64,f64,f64,f64,f64
18853.0,18849.0,18864.0,18849.0,9,2019-12-02 20:00:00.001 EST,3600,6,3,0.0,0.0,0.000833,0.001667,3,9,-3,-0.333333,0.999417,1.000212,1.000212,0.0
18843.0,18843.0,18868.0,18850.0,10,2019-12-02 21:08:28 EST,4107,4,6,0.000053,1.2918e-8,0.001461,0.000974,-2,10,2,0.2,0.998675,1.0,0.999629,0.28
18850.0,18850.0,18875.0,18875.0,10,2019-12-02 22:13:16 EST,3887,5,5,0.001326,3.4120e-7,0.001286,0.001286,0,10,0,0.0,0.998675,1.0,0.998675,1.0
18863.0,18860.0,18873.0,18860.0,10,2019-12-02 23:01:50 EST,2206,6,4,-0.000795,-3.6025e-7,0.001813,0.00272,2,10,-2,-0.2,0.99947,1.000159,1.000159,0.0
18860.0,18858.0,18862.0,18858.0,10,2019-12-03 00:41:30 EST,5929,9,1,-0.000106,-1.7886e-8,0.000169,0.001518,8,10,-8,-0.8,0.999894,1.000106,1.000106,0.0
18867.0,18867.0,18881.0,18879.0,10,2019-12-03 01:12:17 EST,1157,5,5,0.001114,9.6248e-7,0.004322,0.004322,0,10,0,0.0,0.999259,1.0,0.999364,0.857143
18868.0,18860.0,18872.0,18860.0,10,2019-12-03 01:27:35 EST,807,5,5,-0.001006,-0.000001,0.006196,0.006196,0,10,0,0.0,0.999788,1.000424,1.000424,0.0
18858.0,18858.0,18878.0,18878.0,8,2019-12-03 01:49:25.001 EST,1241,4,4,0.000954,7.6906e-7,0.003223,0.003223,0,8,0,0.0,0.998941,1.0,0.998941,1.0
18878.0,18873.0,18882.0,18873.0,12,2019-12-03 01:55:03 EST,337,11,1,-0.000265,-7.8593e-7,0.002967,0.032641,10,12,-10,-0.833333,0.999788,1.000265,1.000265,0.0


In [25]:
BarFeatureSupplier(bar_supplier_ten)

In [27]:
tick_supplier.data.to_struct('a').to_frame()

a
struct[5]
"{2019-12-02 19:00:00 EST,18853.0,1,1,2019-12-02 19:00:00 EST}"
"{2019-12-02 19:01:01 EST,18854.0,1,1,2019-12-02 19:01:01 EST}"
"{2019-12-02 19:11:00 EST,18857.0,1,0,2019-12-02 19:11:00 EST}"
"{2019-12-02 19:12:00 EST,18857.0,1,0,2019-12-02 19:12:00 EST}"
"{2019-12-02 19:17:11 EST,18856.0,1,0,2019-12-02 19:17:11 EST}"
"{2019-12-02 19:32:21 EST,18859.0,1,0,2019-12-02 19:32:21 EST}"
"{2019-12-02 19:33:34 EST,18864.0,1,1,2019-12-02 19:33:34 EST}"
"{2019-12-02 20:00:00 EST,18850.0,1,0,2019-12-02 20:00:00 EST}"
"{2019-12-02 20:00:00.001 EST,18849.0,1,0,2019-12-02 20:00:00.001 EST}"


In [20]:
bar_supplier.data.pivot(index="bar", values=bar_supplier.data.columns)

TypeError: DataFrame.pivot() missing 1 required positional argument: 'columns'

In [18]:
tick_supplier.data

AttributeError: 'NoneType' object has no attribute 'data'

In [29]:
import polars as pl

In [33]:
pl.Categorical

polars.datatypes.Categorical

In [19]:
from datetime import datetime
import polars as pl

class Bar:
    TS = "timestamp"
    OPEN = "open"
    CLOSE = "close"

def create_struct(df):
    return df.with_columns(
        [
            pl.struct(
                ["instrument", col]
            ).alias(pl.select(df)[0, "instrument"] + f"_{col}")
            for col in [Bar.CLOSE, Bar.OPEN]
        ]
    )

# APPL
df_a = pl.DataFrame({
    Bar.TS: [
        datetime(2020, 1, 1, 9, 1, 0),
        datetime(2020, 1, 1, 9, 1, 0),
        datetime(2020, 1, 1, 9, 3, 0),
    ],
    Bar.OPEN: [1, 2, 3],
    Bar.CLOSE: [1, 2, 3],
}
).with_column(
    pl.lit("APPL").alias("instrument").cast(pl.Categorical)
)
#.pipe(create_struct)


# GOOGL
df_b = pl.DataFrame({
    Bar.TS: [
        datetime(2020, 1, 1, 9, 1, 0),
        datetime(2020, 1, 1, 9, 1, 0),
        datetime(2020, 1, 1, 9, 3, 0),
    ],
    Bar.OPEN: [1, 2, 3],
    "foo": [1, 2, 3],
}).with_column(
    pl.lit("GOOGL").alias("instrument").cast(pl.Categorical)
)
#.pipe(create_struct)


In [22]:
df_a.join_asof(df_b, on=Bar.TS)

timestamp,open,close,instrument,open_right,foo,instrument_right
datetime[μs],i64,i64,cat,i64,i64,cat
2020-01-01 09:01:00,1,1,"""APPL""",2,2,"""GOOGL"""
2020-01-01 09:01:00,2,2,"""APPL""",2,2,"""GOOGL"""
2020-01-01 09:03:00,3,3,"""APPL""",3,3,"""GOOGL"""


In [79]:
{k: f"{instrument}_k" for k in df_b.columns}

{'timestamp': 'GOOGL_k',
 'open': 'GOOGL_k',
 'close': 'GOOGL_k',
 'instrument': 'GOOGL_k'}

In [83]:
df = df_b
instrument = "GOOGL"
df.rename({k: f"{instrument}_{k}" for k in df.columns})

GOOGL_timestamp,GOOGL_open,GOOGL_close,GOOGL_instrument
datetime[μs],i64,i64,cat
2020-01-01 09:01:00,1,1,"""GOOGL"""
2020-01-01 09:01:00,2,2,"""GOOGL"""
2020-01-01 09:03:00,3,3,"""GOOGL"""


In [82]:
{k: f"{instrument}_k" for k in df.columns}

{'timestamp': 'GOOGL_k',
 'open': 'GOOGL_k',
 'close': 'GOOGL_k',
 'instrument': 'GOOGL_k'}

In [ ]:
1970-01-01, {"instrument": "BTCUSD", Bar.CLOSE: 4123 }
1970-01-01, {"instrument": "ETHUSD", Bar.CLOSE: 123 }

In [73]:
df_b

timestamp,open,close,instrument,GOOGL_close,GOOGL_open
datetime[μs],i64,i64,cat,struct[2],struct[2]
2020-01-01 09:01:00,1,1,"""GOOGL""","{""GOOGL"",1}","{""GOOGL"",1}"
2020-01-01 09:01:00,2,2,"""GOOGL""","{""GOOGL"",2}","{""GOOGL"",2}"
2020-01-01 09:03:00,3,3,"""GOOGL""","{""GOOGL"",3}","{""GOOGL"",3}"


NotFoundError: GOOGL_TS

In [57]:
df_a.with_column(
    pl.struct(
        ["instrument", Bar.CLOSE]
    ).alias(pl.select(df_a)[0, "instrument"] + f"_{Bar.CLOSE}")
)

open,close,instrument,APPL_close
i64,i64,cat,struct[2]
1,1,"""APPL""","{""APPL"",1}"
2,2,"""APPL""","{""APPL"",2}"
3,3,"""APPL""","{""APPL"",3}"


In [43]:
df_a.with_column(pl.struct(['A','B']).alias('C'))

NotFoundError: A

In [ ]:
df.with_column(pl.lit(1).alias("constant_column")))